# Getting started with Azure ML Data Prep SDK



Wonder how you can make the most of the Azure ML Data Prep SDK? In this "Getting Started" guide, we'll demonstrate how to do your normal data wrangling with this SDK and showcase a few highlights that make this SDK shine. Using a sample of this [Kaggle crime dataset](https://www.kaggle.com/currie32/crimes-in-chicago/home) as an example, we'll cover how to:

* [Read in data](#Read)
* [Profile your data](#Profile)
* [Append rows](#Append)
* [Apply common data science transforms](#Data-science-transforms)
    * [Summarize](#Summarize)
    * [Join](#Join)
    * [Filter](#Filter)
    * [Replace](#Replace)
* [Consume your cleaned dataset](#Consume)
* [Explore advanced features](#Explore)


In [ ]:
from IPython.display import display
from os import path
from tempfile import mkdtemp

import pandas as pd
import azureml.dataprep as dprep

# Paths for datasets
file_crime_dirty  = '../../data/crime-dirty.csv'
file_crime_spring = '../../data/crime-spring.csv'
file_crime_winter = '../../data/crime-winter.csv'
file_aldermen     = '../../data/chicago-aldermen-2015.csv'

# Seed
RAND_SEED = 7251

<a id="Read"></a>

## Read in data

Azure ML Data Prep supports many different file reading formats (i.e. CSV, Excel, Parquet) and the ability to infer column types automatically. To see how powerful the `auto_read_file` capability is, let's take a peek at the `dirty-crime.csv`:

In [ ]:
dprep.read_csv(path=file_crime_dirty).head(7)

A common occurrence in many datasets is to have a column of values with commas; in our case, the last column represents location in the form of longitude-latitude pair. The default CSV reader interprets this comma as a delimiter and thus splits the data into two columns. Furthermore, it incorrectly reads in the header as the column name. Normally, we would need to `skip` the header and specify the delimiter as `|`, but our `auto_read_file` eliminates that work:

In [ ]:
crime_dirty = dprep.auto_read_file(path=file_crime_dirty)

crime_dirty.head(5)

__Advanced features:__ if you'd like to specify the file type and adjust how you want to read files in, you can see the list of our specialized file readers and how to use them [here](../../how-to-guides/data-ingestion.ipynb).

<a id="Profile"></a>

## Profile your data

Let's understand what our data looks like. Azure ML Data Prep facilitates this process by offering data profiles that help us glimpse into column types and column summary statistics. Notice that our auto file reader automatically guessed the column type:

In [ ]:
crime_dirty.get_profile()

<a id="Append"></a>

## Append rows

What if your data is split across multiple files? We support the ability to append multiple datasets column-wise and row-wise. Here, we demonstrate how you can coalesce datasets row-wise:

In [ ]:
# Datasets with the same schema as crime_dirty
crime_winter = dprep.auto_read_file(path=file_crime_winter)
crime_spring = dprep.auto_read_file(path=file_crime_spring)

In [ ]:
crime = (crime_dirty.append_rows(dataflows=[crime_winter, crime_spring]))

crime.take_sample(probability=0.25, seed=RAND_SEED).head(5)

__Advanced features:__ you can learn how to append column-wise and how to deal with appending data with different schemas [here](../../how-to-guides/append-columns-and-rows.ipynb).

<a id="Data-science-transforms"></a>

## Apply common data science transforms

Azure ML Data Prep supports almost all common data science transforms found in other industry-standard data science libraries. Here, we'll explore the ability to `summarize`, `join`, `filter`, and `replace`. 

__Advanced features:__
* We also provide "smart" transforms not found in pandas that use machine learning to [derive new columns](../../how-to-guides/derive-column-by-example.ipynb), [split columns](../../how-to-guides/split-column-by-example.ipynb), and [fuzzy grouping](../../how-to-guides/fuzzy-group.ipynb).
* Finally, we also help featurize your dataset to prepare it for machine learning; learn more about our featurizers like [one-hot encoder](../../how-to-guides/one-hot-encoder.ipynb), [label encoder](../../how-to-guides/label-encoder.ipynb), [min-max scaler](../../how-to-guides/min-max-scaler.ipynb), and [random (train-test) split](../../how-to-guides/random-split.ipynb).
* Our complete list of example Notebooks for transforms can be found in our [How-to Guides](../../how-to-guides).

<a id="Summarize"></a>

### Summarize

Let's see which wards had the most crimes in our sample dataset:

In [ ]:
crime_summary = (crime
    .summarize(
        summary_columns=[
            dprep.SummaryColumnsValue(
                column_id='ID', 
                summary_column_name='total_ward_crimes', 
                summary_function=dprep.SummaryFunction.COUNT
            )
        ],
        group_by_columns=['Ward']
    )
)

(crime_summary
     .sort(sort_order=[('total_ward_crimes', True)])
     .head(5)
)

<a id="Join"></a>

### Join

Let's annotate each observation with more information about the ward where the crime occurred. Let's do so by joining `crime` with a dataset which lists the current aldermen for each ward:

In [ ]:
aldermen = dprep.auto_read_file(path=file_aldermen)

aldermen.head(5)

In [ ]:
crime.join(
    left_dataflow=crime,
    right_dataflow=aldermen,
    join_key_pairs=[
        ('Ward', 'Ward')
    ]
).head(5)

__Advanced features:__ [Learn more](../../how-to-guides/join.ipynb) about how you can do all variants of `join`, like inner-, left-, right-, anti-, and semi-joins.

<a id="Filter"></a>

### Filter

Let's look at theft crimes:

In [ ]:
theft = crime.filter(crime['Primary Type'] == 'THEFT')

theft.head(5)

<a id="Replace"></a>

### Replace

Notice that our `theft` dataset has empty strings in column `Location`. Let's replace those with a missing value:

In [ ]:
theft_replaced = (theft
    .replace_na(
        columns=['Location'], 
        use_empty_string_as_na=True
    )
)

theft_replaced.head(5)

__Advanced features:__ [Learn more](../../how-to-guides/replace-fill-error.ipynb) about more advanced `replace` and `fill` capabilities.

<a id="Consume"></a>

## Consume your cleaned dataset

Azure ML Data Prep allows you to "choose your own adventure" once you're done wrangling. You can:

1. Write to a pandas dataframe
2. Execute on Spark
3. Consume directly in Azure Machine Learning models

In this quickstart guide, we'll show how you can export to a pandas dataframe.

__Advanced features:__ 
* One of the beautiful features of Azure ML Data Prep is that you only need to write your code once and choose whether to scale up or out.
* You can directly consume your new DataFlow in model builders like Azure Machine Learning's [automated machine learning](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/dataprep/auto-ml-dataprep.ipynb).

In [ ]:
theft_replaced.to_pandas_dataframe()

<a id="Explore"></a>

## Explore advanced features

Congratulations on finishing your introduction to the Azure ML Data Prep SDK! If you'd like more detailed tutorials on how to construct machine learning datasets or dive deeper into all of its functionality, you can find more information in our detailed notebooks [here](https://github.com/Microsoft/PendletonDocs). There, we cover topics including how to:

* [Cache your Dataflow to speed up your iterations](../../how-to-guides/cache.ipynb)
* [Add your custom Python transforms](../../how-to-guides/custom-python-transforms.ipynb)
* [Impute missing values](../../how-to-guides/impute-missing-values.ipynb)
* [Sample your data](../../how-to-guides/subsetting-sampling.ipynb)
* [Reference and link between Dataflows](../../how-to-guides/join.ipynb)

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/tutorials/getting-started/getting-started.png)